In [1]:
import numpy as np
import pandas as pd
from scipy.stats import laplace
import scipy.stats
import psycopg2
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
param_dic = {
    "host"      : "localhost",
    "database"  : "bachelorBesoeg2014",
    "user"      : "postgres",
    "password"  : "password",
    "port"      : "5432"
}

def execQuery(params_dic,query):
    try:
        connection = psycopg2.connect(**params_dic)
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")
#"""select count(*) as number_of_dates, count(DISTINCT  count_) as distinct_count from _775147;"""
query = """select time_ from _775147;"""
result = execQuery(param_dic, query)
#print(result)
print(type(result))
print(type(result[0]))
print((str(result[0][0])))
dates = [str(date[0]) for date in result]
values = np.arange(0,len(dates))
zip_iterator = zip(dates, values)

Executed query and closed connection.
<class 'list'>
<class 'tuple'>
2014-01-02


In [3]:
query = """select count_ from _775147;"""
result = execQuery(param_dic, query)
"""
print(result)
print(type(result))
print(type(result[0]))
print((str(result[0][0])))
"""
counts = [(count[0]) for count in result]

Executed query and closed connection.


In [4]:
#counts

In [5]:
#dates

In [6]:
def laplace_mechanism(answear,epsilon):
    b = 1/epsilon
    laplace_mec = laplace(scale=b)
    Y = laplace_mec.rvs()
    return answear + Y

In [7]:
def con_obs(T, epsilon):
    zeta = (1+np.log2(T))/epsilon
    count = laplace(scale=zeta).rvs()
    print(f'zeta={zeta}')
    print(f'count={count}')
    laplace_mec = laplace(scale=zeta)
    print(laplace_mec.var())
    print(laplace_mec.rvs())
    
    print(laplace(scale=zeta).rvs(11))

    
T = len(dates)
epsilon = 0.7
con_obs(T,epsilon)  

zeta=16.86994882131323
count=-13.001388382540172
569.1903464674552
-0.42314377259554037
[ 12.85825516 -16.57305377  -9.61920896   1.25924836 -36.57754956
 -19.59665119  -4.55639925  53.22922718   4.81483957  -6.57226769
 -26.52587256]


In [8]:
T = 1794
print(np.log2(1794))
print(int(np.ceil(np.log2(1794))))
print(2**np.ceil(np.log2(1794)))


10.80896417491926
11
2048.0


In [9]:
class con_obs:
    
    def __init__(self, T, epsilon, dates, counts):
        
        self.T = T
        self.epsilon = epsilon
        self.zeta = (np.log2(T))/epsilon
        self.dates = dates
        
        values = np.arange(0,len(dates))
        zip_iterator = zip(dates, values)
        self.idx_dict =  dict(zip_iterator)
        
        self.real_counts = counts
        
        self.noise_counts = self.__process(self.real_counts)
        
        
    
    def __process(self, dates):
        noise_counts = np.zeros(len(dates))
        
        for idx, date_count in enumerate(counts):
            laplace_sum = np.sum(laplace(scale=self.zeta).rvs(int(np.ceil(np.log2(1794)))))
            noise_counts[idx] = date_count + laplace_sum + noise_counts[idx-1]
            
        return noise_counts
    
    def answer(self, dates):
        
        if len(dates) < 2:
            idx = self.idx_dict[dates[0]]
            #Index is not 0
            if idx:
                return self.noise_counts[idx] - self.noise_counts[idx-1]            
            else:
                return self.noise_counts[idx]
        else:
            """
            print(self.noise_counts[0:4])
            print(self.noise_counts[self.idx_dict[dates[0]]])
            print(self.noise_counts[self.idx_dict[dates[1]]] )
            """
            diff = self.noise_counts[self.idx_dict[dates[1]]+1]  - self.noise_counts[self.idx_dict[dates[0]]]
            return diff
        
    def real_answer(self, dates):
        if len(dates) < 2:
            return self.real_counts[self.idx_dict[dates[0]]]
        else:
            sum_ = np.sum(self.real_counts[self.idx_dict[dates[0]]: self.idx_dict[dates[1]]+1])  
            return sum_
        

In [10]:
T = len(dates)
epsilon = 0.7
c_o = con_obs(T, epsilon, dates, counts)

In [11]:
print(c_o.answer(('2014-01-02','2014-01-05')))
print(c_o.real_answer(('2014-01-02','2014-01-05')))

print(c_o.answer(('2014-01-20',)))
print(c_o.real_answer(('2014-01-20',)))


648.2225705369397
488
197.43208584767262
315


In [60]:
def get_index(date_idx, n_layers):
    idx = []
    print(np.arange(0,n_layers))
    for i in np.arange(0,n_layers):
        if i == 0:
            idx.append(date_idx)
        else:
            idx.append(idx[i-1]//2)

    return idx

In [58]:
T = len(dates)
print(T)
T = 2**np.ceil(np.log2(T))
print(T)
n_layers = int(np.log2(T))
print(f'n_layers = {n_layers}')


1794
2048.0
n_layers = 11


In [63]:
print(get_index(100,n_layers))

[ 0  1  2  3  4  5  6  7  8  9 10]
[100, 50, 25, 12, 6, 3, 1, 0, 0, 0, 0]


In [59]:
print(get_index(0,n_layers))

[ 0  1  2  3  4  5  6  7  8  9 10]
0
[]
First time
1
[0]
2
[0, 0]
3
[0, 0, 0]
4
[0, 0, 0, 0]
5
[0, 0, 0, 0, 0]
6
[0, 0, 0, 0, 0, 0]
7
[0, 0, 0, 0, 0, 0, 0]
8
[0, 0, 0, 0, 0, 0, 0, 0]
9
[0, 0, 0, 0, 0, 0, 0, 0, 0]
10
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
np.log2(T)

11.0

In [77]:
print(2**np.ceil(0))

1.0


In [74]:
print( type(laplace(scale=(np.log2(T))/epsilon).rvs(int(np.ceil(np.log2(1794))))))
print( laplace(scale=(np.log2(T))/epsilon).rvs(int(np.ceil(np.log2(1794)))))
print( type(laplace(scale=(np.log2(T))/epsilon).rvs(1)))
print( laplace(scale=(np.log2(T))/epsilon).rvs(1))

<class 'numpy.ndarray'>
[  1.7930495    0.72489034  -9.10081171  42.02193462  31.71046265
 -10.11842026  55.64229908 -20.92004182  10.7209528   25.73483589
   8.21784395]
<class 'numpy.ndarray'>
[27.88607276]


In [70]:
class con_obs:
    
    def __init__(self, T, epsilon, dates, counts):
        
        self.T = T
        self.epsilon = epsilon
        self.zeta = (np.log2(T))/epsilon
        self.dates = dates
        
        self.n_layers = int(np.log2(T))
        
        self.laplaces = self.init_laplace()
        
        values = np.arange(0,len(dates))
        zip_iterator = zip(dates, values)
        self.idx_dict =  dict(zip_iterator)
        
        self.real_counts = counts
        
        self.noise_counts = self.__process(self.real_counts)
        
        
    def init_laplace(self):
        laplaces = []
        for i in np.arange(1,self.n_layers+1):
            rvs = laplace(scale=self.zeta).rvs(int(np.ceil(np.log2(1794))))
            laplaces.append
        
        return laplaces
    
    
    def __process(self, dates):
        noise_counts = np.zeros(len(dates))
        for idx, date_count in enumerate(counts):
            laplace_sum = np.sum(laplace(scale=self.zeta).rvs(int(np.ceil(np.log2(1794)))))
            noise_counts[idx] = date_count + laplace_sum + noise_counts[idx-1]
        return noise_counts
    
    def answer(self, dates):
        
        if len(dates) < 2:
            idx = self.idx_dict[dates[0]]
            #Index is not 0
            if idx:
                return self.noise_counts[idx] - self.noise_counts[idx-1]            
            else:
                return self.noise_counts[idx]
        else:
            """
            print(self.noise_counts[0:4])
            print(self.noise_counts[self.idx_dict[dates[0]]])
            print(self.noise_counts[self.idx_dict[dates[1]]] )
            """
            diff = self.noise_counts[self.idx_dict[dates[1]]+1]  - self.noise_counts[self.idx_dict[dates[0]]]
            return diff
        
    def real_answer(self, dates):
        if len(dates) < 2:
            return self.real_counts[self.idx_dict[dates[0]]]
        else:
            sum_ = np.sum(self.real_counts[self.idx_dict[dates[0]]: self.idx_dict[dates[1]]+1])  
            return sum_

        
T = len(dates)
epsilon = 0.7
c_o = con_obs(T, epsilon, dates, counts)

i = 0
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
